# Stabiliseur 

In [11]:
import cv2
from vidstab import VidStab

def stabilize_video(input_path, output_path):
    # Initialize video stabilizer
    stabilizer = VidStab()
    
    # Open the input video
    vid = cv2.VideoCapture(input_path)
    
    # Get video properties
    width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(vid.get(cv2.CAP_PROP_FPS))
    
    # Create VideoWriter object
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    
    while True:
        # Read a frame from the input video
        ret, frame = vid.read()
        
        if not ret:
            break
        
        # Stabilize the frame
        stabilized_frame = stabilizer.stabilize_frame(input_frame=frame, border_type='black')
        
        # Write the stabilized frame to the output video
        out.write(stabilized_frame)
    
    # Release video objects
    vid.release()
    out.release()
    cv2.destroyAllWindows()
    
    print("Video stabilization complete!")

# Example usage
input_video = 'input_video.mp4'
output_video = 'stabilized_video_V1.mp4'

stabilize_video(input_video, output_video)



Video stabilization complete!


# Stabiliseur avec editing

In [12]:
import cv2
import numpy as np
from vidstab import VidStab

def stabilize_video(input_path, output_path):
    # Initialize video stabilizer
    stabilizer = VidStab()
    
    # Open the input video
    vid = cv2.VideoCapture(input_path)
    
    # Get video properties
    width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(vid.get(cv2.CAP_PROP_FPS))
    
    # Stabilize the entire video first to calculate the crop region
    stabilized_frames = []
    while True:
        ret, frame = vid.read()
        if not ret:
            break
        stabilized_frame = stabilizer.stabilize_frame(input_frame=frame, border_type='black')
        stabilized_frames.append(stabilized_frame)
    
    # Calculate the maximum movement in all directions
    max_left = max_right = max_top = max_bottom = 0
    for frame in stabilized_frames:
        non_black = np.where(frame != 0)
        if len(non_black[0]) > 0 and len(non_black[1]) > 0:
            top, bottom = non_black[0].min(), non_black[0].max()
            left, right = non_black[1].min(), non_black[1].max()
            max_top = max(max_top, top)
            max_bottom = min(max_bottom, bottom) if max_bottom != 0 else bottom
            max_left = max(max_left, left)
            max_right = min(max_right, right) if max_right != 0 else right
    
    # Calculate new dimensions
    new_height = max_bottom - max_top
    new_width = max_right - max_left
    
    # Ensure new dimensions are positive
    if new_height <= 0 or new_width <= 0:
        print("Error: Stabilization resulted in invalid dimensions. Using original dimensions.")
        new_height, new_width = height, width
        max_top, max_bottom, max_left, max_right = 0, height, 0, width
    
    # Create VideoWriter object with new dimensions
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (new_width, new_height))
    
    # Process and write the stabilized frames
    for frame in stabilized_frames:
        cropped_frame = frame[max_top:max_bottom, max_left:max_right]
        # Resize if necessary
        if cropped_frame.shape[:2] != (new_height, new_width):
            cropped_frame = cv2.resize(cropped_frame, (new_width, new_height))
        out.write(cropped_frame)
    
    # Release video objects
    vid.release()
    out.release()
    cv2.destroyAllWindows()
    
    print(f"Video stabilization complete! New dimensions: {new_width}x{new_height}")

# Example usage
input_video = 'input_video.mp4'
output_video = 'stabilized_video_V2.mp4'

stabilize_video(input_video, output_video)


Video stabilization complete! New dimensions: 350x629
